# Path

In [12]:
import sys
import os

project_path = r'/home/craxiss/Documents/projects/kapsul_etkinlik_v2'

os.chdir(project_path)
sys.path.append(project_path)

# Imports

In [13]:
from helpers import column_formatter, utf8_convert, first_letter_id_generator
from charset_normalizer import detect
from config import data_path as path
from string import ascii_lowercase
from json import load, dump
import pandas as pd
import openpyxl

# Read

In [14]:
information = pd.read_excel(
    path+'/3-result/information.xlsx', index_col='Unnamed: 0')
events_per_person = pd.read_excel(
    path+'/3-result/events_per_person.xlsx', index_col='Unnamed: 0')

events = pd.read_excel(
    path+'/3-result/events.xlsx', index_col='Unnamed: 0')

with pd.ExcelFile(path + '/2-cleaned/basvuru.xlsx') as appeals_xlsx:
    appeals = pd.read_excel(
        appeals_xlsx, index_col='Unnamed: 0', sheet_name='Başvurular')
    lessons_all = pd.read_excel(
        appeals_xlsx, index_col='Unnamed: 0', sheet_name='Dersler')


with pd.ExcelFile(path + '/2-cleaned/yoklama.xlsx') as rolls_xlsx:
    rolls_info = pd.read_excel(
        rolls_xlsx, 'General Info', index_col='Unnamed: 0')

    rolls = dict()
    for sheet in rolls_info.index:
        rolls[sheet] = pd.read_excel(rolls_xlsx, sheet)

information['id'] = information.index


# One Hot Encoding

In [15]:
events_list = list(events['eğitim'])

In [16]:
cols = list()
list(map(lambda x: cols.extend([x + " BAŞVURU", x + " KATILIM"]), events_list))
ohe = pd.DataFrame(columns = cols)

In [17]:
for row in events_per_person.iterrows():
    ohe_row = dict.fromkeys(cols, 0)
    index, row = row[0], row[1]
    appeals, rolls = row[1], row[2]

    if type(appeals) != float:
        appeals = appeals.split(',')

        for appeal in appeals:
            ohe_row[appeal + ' BAŞVURU'] = 1


    if type(rolls) != float:
        rolls = rolls.split(',')

        for appeal in rolls:
            ohe_row[appeal + ' KATILIM'] = 1

    ohe = pd.concat([ohe, pd.DataFrame([ohe_row], index=[index])])

# Column Hashing

In [18]:
ohe.columns = list(map(lambda x: f'b{(x[0]/2) + 1:.0f}' if('BAŞVURU' in x[1]) else f'k{((x[0]-1)/2) + 1:.0f}', enumerate(cols)))

# Write

In [19]:
ohe.to_excel(path+'/3-result/event_per_person_ohe.xlsx')